In [111]:
import datetime
import pandas as pd
import numpy as np
import yfinance as yf


# for plotting purposes
from datetime import datetime
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [112]:
# Equivalent timeframe mappings
frame_M1 = '1m'
frame_M15 = '15m'
frame_M30 = '30m'
frame_H1 = '1h'
frame_H4 = '4h'
frame_D1 = '1d'
frame_W1 = '1wk'


# Matching your MT5 asset list with Yahoo tickers
assets = {
    'EURUSD': 'EURUSD=X',
    'USDCHF': 'USDCHF=X',
    'GBPUSD': 'GBPUSD=X',
    'USDCAD': 'USDCAD=X',
    'BTCUSD': 'BTC-USD',
    'ETHUSD': 'ETH-USD',
    'XAUUSD': 'XAUUSD=X',
    'XAGUSD': 'XAGUSD=X',
    'SP500m': '^GSPC',
    'UK100': '^FTSE'
}

In [113]:
# SUPPORT AND RESISTANCE

def get_quotes(time_frame,  period='2yrs', asset='EURUSD'):
    try:
        symbol = assets[asset]
        df = yf.download(
            symbol,
            period=period,
            interval=time_frame,
        )

        if df.empty:
            print(f"No data for {asset}. Check ticker symbol or timeframe.")
            return pd.DataFrame()

        df.reset_index(inplace=True)
        return df

    except Exception as e:
        print(f"Error fetching data for {asset}: {e}")
        return pd.DataFrame()


def mass_import(asset_index, time_frame):

    asset_list = list(assets.keys())
    asset = asset_list[asset_index]

    frame_M15 = '15m'
    frame_M30 = '30m'
    frame_H1 = '1h'
    frame_H4 = '4h'
    frame_D1 = '1d'
    frame_W1 = '1wk'

    if time_frame == '1WK':
        data = get_quotes(frame_W1, '10y', asset=asset)
    elif time_frame == 'D1':
        data = get_quotes(frame_D1, '5y', asset=asset)
    elif time_frame == '4H':
        data = get_quotes(frame_H4, '2y', asset=asset)
    elif time_frame == '1H':
        data = get_quotes(frame_H1, '2y', asset=asset)
    elif time_frame == 'M30':
        data = get_quotes(frame_M30, '60d', asset=asset)
    elif time_frame == 'M15':
        data = get_quotes(frame_M15, '60d', asset=asset)
    else:
        data = get_quotes(frame_D1, '5y', asset=asset)

    if data.empty:
        print(f"No data found for {asset} on timeframe {time_frame}")
        return np.array([])

    return data


def detect_key_levels(df, current_candle, backcandles=50, test_candles=10):
    """
    Detects key support and resistance levels in a given backcandles window.
    
    A level is identified if a candle's high is the highest or its low is the lowest 
    compared to `test_candles` before and after it.

    Parameters:
        df (pd.DataFrame): DataFrame containing 'High' and 'Low' columns.
        current_candle (int): The index of the current candle (latest available candle).
        backcandles (int): Number of candles to look back.
        test_candles (int): Number of candles before and after each candle to check.

    Returns:
        dict: A dictionary with detected 'support' and 'resistance' levels.
    """
    key_levels = {"support": [], "resistance": []}

    # Define the last candle that can be tested to avoid lookahead bias
    last_testable_candle = current_candle - test_candles

    # Ensure we have enough data
    if last_testable_candle < backcandles + test_candles:
        return key_levels  # Not enough historical data

    # Iterate through the backcandles window
    for i in range(current_candle - backcandles, last_testable_candle):
        high = df['High'].iloc[i]
        low = df['Low'].iloc[i]

        # Get surrounding window of test_candles before and after
        before = df.iloc[max(0, i - test_candles):i]
        after = df.iloc[i + 1: min(len(df), i + test_candles + 1)]

        # Check if current high is the highest among before & after candles
        if high > before['High'].max() and high > after['High'].max():
            key_levels["resistance"].append((i, high))

        # Check if current low is the lowest among before & after candles
        if low < before['Low'].min() and low < after['Low'].min():
            key_levels["support"].append((i, low))

    return key_levels


def fill_key_levels(df, backcandles=50, test_candles=10):
    """
    Adds a 'key_levels' column to the DataFrame where each row contains all
    key support and resistance levels detected up to that candle (including
    both the level value and the index of the candle that generated it).
    
    Parameters:
        df (pd.DataFrame): DataFrame containing 'High' and 'Low' columns.
        backcandles (int): Lookback window for detecting key levels.
        test_candles (int): Number of candles before/after for validation.

    Returns:
        pd.DataFrame: Updated DataFrame with the new 'key_levels' column.
    """
    df["key_levels"] = None  # Initialize the column

    from tqdm import tqdm
    for current_candle in tqdm(range(backcandles + test_candles, len(df))):
        # Detect key levels for the current candle
        key_levels = detect_key_levels(
            df, current_candle, backcandles, test_candles)

        # Collect support and resistance levels (with their indices) up to current_candle
        support_levels = [(idx, level) for (idx, level) in key_levels["support"]
                          if idx < current_candle]
        resistance_levels = [(idx, level) for (idx, level) in key_levels["resistance"]
                             if idx < current_candle]

        # Store the levels along with the originating candle index
        if support_levels or resistance_levels:
            df.at[current_candle, "key_levels"] = {
                "support": support_levels,
                "resistance": resistance_levels
            }

    return df


def flatten_yf_columns(df: pd.DataFrame) -> pd.DataFrame:
    """
    Removes the 'Ticker' row level from a Yahoo Finance DataFrame
    that has multi-index columns like ('EURUSD=X', 'Open').

    Returns a DataFrame with simple column names like 'Open', 'High', etc.
    """
    # If columns are MultiIndex (e.g. ('EURUSD=X', 'Open')), flatten them
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.droplevel(1)
    return df


def detect_fvg(data, lookback_period=14, body_multiplier=1.5, min_gap_val=0.2):
    """
    Detects Fair Value Gaps (FVGs) in historical price data.

    Rules Added:
    - Middle candle body must be > avg body * body_multiplier
    - The FVG gap must be > 10% of the middle candle body
    """

    fvg_list = [None, None]

    for i in range(2, len(data)):
        first_high = data['High'].iloc[i-2]
        first_low = data['Low'].iloc[i-2]
        middle_open = data['Open'].iloc[i-1]
        middle_close = data['Close'].iloc[i-1]
        third_low = data['Low'].iloc[i]
        third_high = data['High'].iloc[i]

        # Middle candle body
        middle_body = abs(middle_close - middle_open)

        # Avg body over lookback
        prev_bodies = (data['Close'].iloc[max(0, i-1-lookback_period):i-1] -
                       data['Open'].iloc[max(0, i-1-lookback_period):i-1]).abs()
        avg_body_size = prev_bodies.mean()
        avg_body_size = avg_body_size if avg_body_size > 0 else 0.001

   
        # Minimum required FVGgap size
        min_gap = middle_body * min_gap_val

        # -----------------------------
        # Check Bullish FVG
        # -----------------------------
        if third_low > first_high:
            gap_size = third_low - first_high

            if (middle_body > avg_body_size * body_multiplier
                    and gap_size > min_gap):

                fvg_list.append(('bullish', first_high, third_low, i))
                # print("bullish", first_high, third_low, i)
                continue

        # -----------------------------
        # Check Bearish FVG
        # -----------------------------
        if third_high < first_low:
            gap_size = first_low - third_high

            if (middle_body > avg_body_size * body_multiplier
                    and gap_size > min_gap):

                fvg_list.append(('bearish', first_low, third_high, i))
                # print("bearish", first_low, third_high, i)
                continue

        # No FVG
        fvg_list.append(None)

    return fvg_list

In [114]:
data_15min_raw = mass_import(0, 'M15')
data_30min_raw = mass_import(0, 'M30')
data_1hour_raw = mass_import(0, '1H')
data_4hour_raw = mass_import(0, '4H')
# data_1day_raw = mass_import(0, '1D')
data_1week_raw = mass_import(0, '1WK')

/var/folders/pp/x__1tf4n7j98nw_8bwlmgxj00000gn/T/ipykernel_44839/1640522790.py:6: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  1 of 1 completed
/var/folders/pp/x__1tf4n7j98nw_8bwlmgxj00000gn/T/ipykernel_44839/1640522790.py:6: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  1 of 1 completed
/var/folders/pp/x__1tf4n7j98nw_8bwlmgxj00000gn/T/ipykernel_44839/1640522790.py:6: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  1 of 1 completed
/var/folders/pp/x__1tf4n7j98nw_8bwlmgxj00000gn/T/ipykernel_44839/1640522790.py:6: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  1 of 1 completed
/var/folders/pp/x__1tf4n7j98nw_8bwlmgxj00000gn/T/ipykernel_44839/1640522790.

In [115]:
data_1hour_raw

Price,Datetime,Close,High,Low,Open,Volume
Ticker,,EURUSD=X,EURUSD=X,EURUSD=X,EURUSD=X,EURUSD=X
0,2023-11-20 12:00:00+00:00,1.093255,1.093255,1.092419,1.092538,0
1,2023-11-20 13:00:00+00:00,1.093016,1.093494,1.092777,1.093374,0
2,2023-11-20 14:00:00+00:00,1.094331,1.094691,1.092299,1.093016,0
3,2023-11-20 15:00:00+00:00,1.094092,1.094451,1.092896,1.094451,0
4,2023-11-20 16:00:00+00:00,1.094691,1.094811,1.094092,1.094212,0
...,...,...,...,...,...,...
12376,2025-11-20 08:00:00+00:00,1.152472,1.152605,1.151543,1.152339,0
12377,2025-11-20 09:00:00+00:00,1.153136,1.153136,1.152206,1.152472,0
12378,2025-11-20 10:00:00+00:00,1.153004,1.153270,1.152871,1.153136,0


In [116]:
# Move Datetime from index to column
data_15min = data_15min_raw.reset_index()
data_30min = data_30min_raw.reset_index()
data_1hour = data_1hour_raw.reset_index()
data_4hour = data_4hour_raw.reset_index()
# data_1day = mass_import(0, '1D')
data_1week = data_1week_raw.reset_index() 

4HR CHART WITH FVG


In [117]:
data_4hour.rename(columns={'Date': 'Gmt time'}, inplace=True)


try:
    my_data = data_4hour[['Datetime', 'Open', 'High', 'Low', 'Close', 'Volume']]
except:
    my_data = data_4hour[['Gmt time', 'Open', 'High', 'Low', 'Close', 'Volume']]


data_4hour = flatten_yf_columns(my_data)
data_4hour


data_4hour['FVG'] = detect_fvg(data_4hour, 14, 1.5, 0.2)

data_4hour.tail(20)


dfpl = data_4hour.tail(50)

# Create the figure
fig = go.Figure()

# Add candlestick chart
fig.add_trace(go.Candlestick(
    x=dfpl.index,
    open=dfpl["Open"],
    high=dfpl["High"],
    low=dfpl["Low"],
    close=dfpl["Close"],
    name="Candles"
))

# Add FVG zones
for _, row in dfpl.iterrows():
    if isinstance(row["FVG"], tuple):
        fvg_type, start, end, index = row["FVG"]
        # print('FVG LEVEL *****', row["FVG"])
        color = "rgba(0,255,0,0.3)" if fvg_type == "bullish" else "rgba(255,0,0,0.3)"
        fig.add_shape(
            type="rect",
            x0=index - 2,
            x1=index + 30,
            y0=start,
            y1=end,
            fillcolor=color,
            opacity=0.8,
            layer="below",
            line=dict(width=0),
        )

# Show the chart
fig.update_layout(width=1200, height=800,
                  xaxis=dict(showgrid=False),
                  yaxis=dict(showgrid=False),
                  plot_bgcolor='black',
                  paper_bgcolor='black')
fig.show()

1HR CHART  

In [118]:
data_1hour.rename(columns={'Date': 'Gmt time'}, inplace=True)


try:
    my_data = data_1hour[['Datetime', 'Open',
                          'High', 'Low', 'Close', 'Volume']]
except:
    my_data = data_1hour[['Gmt time', 'Open',
                          'High', 'Low', 'Close', 'Volume']]


data_1hour = flatten_yf_columns(my_data)
data_1hour


data_1hour['FVG'] = detect_fvg(data_1hour, 14, 1.5, 0.2)

data_1hour.tail(20)


dfpl = data_1hour.tail(70)

# Create the figure
fig = go.Figure()

# Add candlestick chart
fig.add_trace(go.Candlestick(
    x=dfpl.index,
    open=dfpl["Open"],
    high=dfpl["High"],
    low=dfpl["Low"],
    close=dfpl["Close"],
    name="Candles"
))

# Add FVG zones
for _, row in dfpl.iterrows():
    if isinstance(row["FVG"], tuple):
        fvg_type, start, end, index = row["FVG"]
        print('FVG LEVEL *****', row["FVG"])
        color = "rgba(0,255,0,0.3)" if fvg_type == "bullish" else "rgba(255,0,0,0.3)"
        fig.add_shape(
            type="rect",
            x0=index - 2,
            x1=index + 30,
            y0=start,
            y1=end,
            fillcolor=color,
            opacity=0.8,
            layer="below",
            line=dict(width=0),
        )

# Show the chart
fig.update_layout(width=1200, height=800,
                  xaxis=dict(showgrid=False),
                  yaxis=dict(showgrid=False),
                  plot_bgcolor='black',
                  paper_bgcolor='black')
fig.show()

FVG LEVEL ***** ('bearish', np.float64(1.1598236560821533), np.float64(1.1587486267089844), 12336)
FVG LEVEL ***** ('bearish', np.float64(1.1574074029922485), np.float64(1.1560693979263306), 12359)
FVG LEVEL ***** ('bearish', np.float64(1.1559356451034546), np.float64(1.155001163482666), 12360)
FVG LEVEL ***** ('bullish', np.float64(1.1531364917755127), np.float64(1.1539349555969238), 12366)
FVG LEVEL ***** ('bearish', np.float64(1.1532695293426514), np.float64(1.1526048183441162), 12370)
FVG LEVEL ***** ('bearish', np.float64(1.1528706550598145), np.float64(1.1519410610198975), 12380)


1HR CHART WITH SUPORT AND RESISTANCE

In [119]:
data_1hour = fill_key_levels(data_1hour, backcandles=50, test_candles=10)

100%|██████████| 12321/12321 [00:25<00:00, 480.88it/s]


In [120]:
data_1hour

Price,Datetime,Open,High,Low,Close,Volume,FVG,key_levels
0,2023-11-20 12:00:00+00:00,1.092538,1.093255,1.092419,1.093255,0,None,None
1,2023-11-20 13:00:00+00:00,1.093374,1.093494,1.092777,1.093016,0,None,None
2,2023-11-20 14:00:00+00:00,1.093016,1.094691,1.092299,1.094331,0,None,None
3,2023-11-20 15:00:00+00:00,1.094451,1.094451,1.092896,1.094092,0,None,None
4,2023-11-20 16:00:00+00:00,1.094212,1.094811,1.094092,1.094691,0,None,None
...,...,...,...,...,...,...,...,...
12376,2025-11-20 08:00:00+00:00,1.152339,1.152605,1.151543,1.152472,0,None,"{'support': [(12337, 1.1575413942337036)], 're..."
12377,2025-11-20 09:00:00+00:00,1.152472,1.153136,1.152206,1.153136,0,None,"{'support': [(12337, 1.1575413942337036)], 're..."
12378,2025-11-20 10:00:00+00:00,1.153136,1.153270,1.152871,1.153004,0,None,"{'support': [(12337, 1.1575413942337036)], 're..."
12379,2025-11-20 11:00:00+00:00,1.153004,1.153004,1.151808,1.151808,0,None,"{'support': [(12337, 1.1575413942337036)], 're..."


In [121]:
def plot_fvg_and_key_levels(df, start_idx, end_idx, extension=20):
    """
    Plots candlesticks, FVG zones, and key levels (support/resistance) for a
    subset of a DataFrame from `start_idx` to `end_idx`.
    
    The FVG column is expected to have tuples of the form:
        (fvg_type, start_price, end_price, trigger_index)

    The key_levels column is expected to have dictionaries of the form:
        {
          "support": [(idx, price), (idx, price), ...],
          "resistance": [(idx, price), (idx, price), ...]
        }

    Parameters:
    -----------
    df : pd.DataFrame
        Must contain: "Open", "High", "Low", "Close", "FVG", "key_levels".
    start_idx : int
        Starting row index for plotting.
    end_idx : int
        Ending row index for plotting.
    extension : int
        How far (in x-axis units/index steps) to extend the FVG rectangles
        and key-level lines.
    
    Returns:
    --------
    fig : plotly.graph_objects.Figure
        A Plotly Figure with the candlesticks, FVG, and key-level lines.
    """

    # Slice the DataFrame to the desired plotting range
    dfpl = df.loc[start_idx:end_idx]

    # Create the figure
    fig = go.Figure()

    # -- 1) Add Candlestick Chart --
    fig.add_trace(go.Candlestick(
        x=dfpl.index,
        open=dfpl["Open"],
        high=dfpl["High"],
        low=dfpl["Low"],
        close=dfpl["Close"],
        name="Candles"
    ))

    # -- 2) Add FVG Zones --
    for i, row in dfpl.iterrows():
        # Check if "FVG" is a valid tuple: (fvg_type, start_price, end_price, trigger_index)
        if isinstance(row.get("FVG"), tuple):
            fvg_type, start_price, end_price, trigger_idx = row["FVG"]

            # Choose a fill color based on bullish vs. bearish
            if fvg_type == "bullish":
                color = "rgba(0, 255, 0, 0.3)"   # greenish
            else:
                color = "rgba(255, 0, 0, 0.3)"   # reddish

            fig.add_shape(
                type="rect",
                x0=trigger_idx,
                x1=trigger_idx + extension,
                y0=start_price,
                y1=end_price,
                fillcolor=color,
                opacity=0.4,
                layer="below",
                line=dict(width=0),
            )

    # -- 3) Add Key Levels as Horizontal Lines --
    for i, row in dfpl.iterrows():
        key_levels = row.get("key_levels", None)
        if key_levels:
            # key_levels is a dict: {"support": [(idx, val), ...], "resistance": [(idx, val), ...]}
            support_levels = key_levels.get("support", [])
            resistance_levels = key_levels.get("resistance", [])

            # Plot support levels
            for (gen_idx, s_price) in support_levels:
                # We only draw the line if gen_idx is in (start_idx, end_idx)
                # You can decide to relax/omit this check if you want lines from outside the window.
                if start_idx <= gen_idx <= end_idx:
                    fig.add_shape(
                        type="line",
                        x0=gen_idx,
                        x1=gen_idx + extension,
                        y0=s_price,
                        y1=s_price,
                        line=dict(color="blue", width=2),
                        layer="below"
                    )

            # Plot resistance levels
            for (gen_idx, r_price) in resistance_levels:
                if start_idx <= gen_idx <= end_idx:
                    fig.add_shape(
                        type="line",
                        x0=gen_idx,
                        x1=gen_idx + extension,
                        y0=r_price,
                        y1=r_price,
                        line=dict(color="orange", width=2),
                        layer="below"
                    )

    # -- 4) Figure Aesthetics --
    fig.update_layout(
        width=1200,
        height=800,
        xaxis=dict(showgrid=False),
        yaxis=dict(showgrid=False),
        plot_bgcolor='black',
        paper_bgcolor='black'
    )

    fig.show()
    return fig


fig = plot_fvg_and_key_levels(
    my_data, start_idx=12180, end_idx=12380, extension=20)

15M CHART FVG

In [ ]:
data_15min.rename(columns={'Date': 'Gmt time'}, inplace=True)


try:
    my_data = data_15min[['Datetime', 'Open',
                          'High', 'Low', 'Close', 'Volume']]
except:
    my_data = data_15min[['Gmt time', 'Open',
                          'High', 'Low', 'Close', 'Volume']]


data_15min = flatten_yf_columns(my_data)
data_15min


data_15min['FVG'] = detect_fvg(data_15min, 14, 1.5, 0.05)

data_15min.tail(20)


dfpl = data_15min.tail(200)

# Create the figure
fig = go.Figure()

# Add candlestick chart
fig.add_trace(go.Candlestick(
    x=dfpl.index,
    open=dfpl["Open"],
    high=dfpl["High"],
    low=dfpl["Low"],
    close=dfpl["Close"],
    name="Candles"
))

# Add FVG zones
for _, row in dfpl.iterrows():
    if isinstance(row["FVG"], tuple):
        fvg_type, start, end, index = row["FVG"]
        # print('FVG LEVEL *****', row["FVG"])
        color = "rgba(0,255,0,0.3)" if fvg_type == "bullish" else "rgba(255,0,0,0.3)"
        fig.add_shape(
            type="rect",
            x0=index - 2,
            x1=index + 30,
            y0=start,
            y1=end,
            fillcolor=color,
            opacity=0.8,
            layer="below",
            line=dict(width=0),
        )

# Show the chart
fig.update_layout(width=1200, height=800,
                  xaxis=dict(showgrid=False),
                  yaxis=dict(showgrid=False),
                  plot_bgcolor='black',
                  paper_bgcolor='black')
fig.show()

FVG LEVEL ***** ('bearish', np.float64(1.1603620052337646), np.float64(1.1587486267089844), 5443)
FVG LEVEL ***** ('bullish', np.float64(1.1584800481796265), np.float64(1.1586142778396606), 5476)
FVG LEVEL ***** ('bearish', np.float64(1.1583458185195923), np.float64(1.1579434871673584), 5483)
FVG LEVEL ***** ('bullish', np.float64(1.158077597618103), np.float64(1.1582117080688477), 5486)
FVG LEVEL ***** ('bullish', np.float64(1.1583458185195923), np.float64(1.1586142778396606), 5490)
FVG LEVEL ***** ('bearish', np.float64(1.1586142778396606), np.float64(1.1583458185195923), 5492)
FVG LEVEL ***** ('bullish', np.float64(1.1584800481796265), np.float64(1.1587486267089844), 5495)
FVG LEVEL ***** ('bullish', np.float64(1.1587486267089844), np.float64(1.1590172052383423), 5498)
FVG LEVEL ***** ('bearish', np.float64(1.1592857837677002), np.float64(1.159151554107666), 5503)
FVG LEVEL ***** ('bearish', np.float64(1.1583458185195923), np.float64(1.158077597618103), 5510)
FVG LEVEL ***** ('beari

Support and Resistance on 1h drawn on 15m chart with FVG

In [123]:
import pandas as pd
import plotly.graph_objects as go
# Assuming data_1hour and data_15min are already loaded DataFrames, and
# 'key_levels' are tuples/lists of (time_or_index, price)


# --- Extract clean S/R levels from 1H and map them by timestamp ---
sr_levels = []

for i, row in data_1hour.iterrows():
    if row["key_levels"] is None:
        continue

    # Use .get() for safe column access
    ts = row.get("Datetime") or row.get("Gmt time")
    if ts is None:  # Skip if no valid timestamp
        continue

    # Support
    for (_, s_price) in row["key_levels"].get("support", []):
        sr_levels.append({
            "timestamp": ts,
            "price": s_price,
            "type": "support"
        })

    # Resistance
    for (_, r_price) in row["key_levels"].get("resistance", []):
        sr_levels.append({
            "timestamp": ts,
            "price": r_price,
            "type": "resistance"
        })

# Convert to DataFrame for easier merge
sr_df = pd.DataFrame(sr_levels)


# Merge S/R into 15m chart
# Initialize columns with empty lists
data_15min["sr_support"] = [[] for _ in range(len(data_15min))]
data_15min["sr_resistance"] = [[] for _ in range(len(data_15min))]

for i, row in data_15min.iterrows():
    # Use .get() for safe column access
    t = row.get("Datetime") or row.get("Gmt time")
    if t is None:  # Skip if no valid timestamp
        continue

    # Filter S/R levels that occurred before or at the current timestamp 't'
    matching = sr_df[sr_df["timestamp"] <= t]

    supports = matching[matching["type"] == "support"]["price"].tolist()
    resistances = matching[matching["type"] == "resistance"]["price"].tolist()

    # Store only the latest 10 levels
    data_15min.at[i, "sr_support"] = supports[-10:]
    data_15min.at[i, "sr_resistance"] = resistances[-10:]


# Use .copy() to avoid SettingWithCopyWarning
dfpl = data_15min.tail(200).copy()

fig = go.Figure()

# 15m Candles
fig.add_trace(go.Candlestick(
    x=dfpl.index,
    open=dfpl["Open"],
    high=dfpl["High"],
    low=dfpl["Low"],
    close=dfpl["Close"],
    name="15m Candles"
))

# --- FVG Zones ---
for _, row in dfpl.iterrows():
    if isinstance(row["FVG"], tuple):
        fvg_type, start, end, index = row["FVG"]
        color = "rgba(0,255,0,0.3)" if fvg_type == "bullish" else "rgba(255,0,0,0.3)"
        fig.add_shape(
            type="rect",
            x0=index - 2,
            x1=index + 30,
            y0=start,
            y1=end,
            fillcolor=color,
            opacity=0.75,
            layer="below",
            line=dict(width=0),
        )


# --- ADD 1-HOUR SUPPORT & RESISTANCE ON 15M CHART (Optimized) ---

# 1. Collect all UNIQUE S/R prices from the visible data
all_supports = set()
all_resistances = set()

for _, row in dfpl.iterrows():
    all_supports.update(row["sr_support"])
    all_resistances.update(row["sr_resistance"])


# 2. Draw each UNIQUE S/R line ONCE
x_min = dfpl.index.min()
x_max = dfpl.index.max() + 50  # Max extent used in original code

# Support
for s_price in all_supports:
    fig.add_shape(
        type="line",
        x0=x_min,
        x1=x_max,
        y0=s_price,
        y1=s_price,
        line=dict(color="blue", width=1),
        layer="below"
    )

# Resistance
for r_price in all_resistances:
    fig.add_shape(
        type="line",
        x0=x_min,
        x1=x_max,
        y0=r_price,
        y1=r_price,
        line=dict(color="orange", width=1),
        layer="below"
    )


fig.update_layout(width=1200, height=800,
                  xaxis=dict(showgrid=False),
                  yaxis=dict(showgrid=False),
                  plot_bgcolor='black',
                  paper_bgcolor='black')

fig.show()